# containers_with_enums_and_table_writer


## Create some example Containers

In [1]:
import enum
from ctapipe.io import HDF5TableWriter
from ctapipe.core import Container, Field
from astropy import units as u
import numpy as np

In [2]:
class WithEnum(Container):
    
    # this class could also be defined in global namespace 
    # outside this container, but this looks a bit tidier.
    # both variants work however
    class EventType(enum.Enum):
        pedestal = 1
        physics = 2
        calibration = 3
    
    event_type = Field(
        EventType.calibration, 
        f'type of event, one of: {list(EventType.__members__.keys())}'
    )

let's also make a dummy stream (generator) that will create a series of these containers

In [3]:
def create_stream(n_event):
    data = WithEnum()
    for i in range(n_event):
        data.event_type = WithEnum.EventType(i % 3 + 1)
        yield data

In [4]:
for data in create_stream(3):
    for key, val in data.items():
        print('{}: {}, type : {}'.format(key, val, type(val)))

event_type: EventType.pedestal, type : <enum 'EventType'>
event_type: EventType.physics, type : <enum 'EventType'>
event_type: EventType.calibration, type : <enum 'EventType'>


## Writing the Data

In [5]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
    for data in create_stream(10):
        h5_table.write('table', data)

In [6]:
!ls container.h5

container.h5


## Reading the Data

In [7]:
import pandas as pd

data = pd.read_hdf('container.h5', key='/data/table')
data.head()

,event_type
0,1
1,2
2,3
3,1
4,2


#### Reading with PyTables

In [8]:
import tables
h5  = tables.open_file('container.h5')
table = h5.root['data']['table']
table

/data/table (Table(10,), fletcher32, shuffle, blosc:zstd(5)) 'Storage of WithEnum'
  description := {
  "event_type": Int64Col(shape=(), dflt=0, pos=0)}
  byteorder := 'little'
  chunkshape := (8192,)

In [9]:
table.attrs

/data/table._v_attrs (AttributeSet), 10 attributes:
   [CLASS := 'TABLE',
    CTAPIPE_VERSION := '0.10.4.dev6+g833cd994',
    FIELD_0_FILL := 0,
    FIELD_0_NAME := 'event_type',
    NROWS := 10,
    TITLE := 'Storage of WithEnum',
    VERSION := '2.7',
    event_type_DESC := "type of event, one of: ['pedestal', 'physics', 'calibration']",
    event_type_ENUM := <enum 'EventType'>,
    event_type_TRANSFORM := 'enum']

In [10]:
from ctapipe.io import HDF5TableReader

def read(mode):
    
    print('reading mode {}'.format(mode))

    with HDF5TableReader('container.h5', mode=mode) as h5_table:

        for group_name in ['data/']:

            group_name = '/{}table'.format(group_name)
            print(group_name)

            for data in h5_table.read(group_name, WithEnum()):

                print(data.as_dict())

In [11]:
read('r')

reading mode r
/data/table
{'event_type': <EventType.pedestal: 1>}
{'event_type': <EventType.physics: 2>}
{'event_type': <EventType.calibration: 3>}
{'event_type': <EventType.pedestal: 1>}
{'event_type': <EventType.physics: 2>}
{'event_type': <EventType.calibration: 3>}
{'event_type': <EventType.pedestal: 1>}
{'event_type': <EventType.physics: 2>}
{'event_type': <EventType.calibration: 3>}
{'event_type': <EventType.pedestal: 1>}
